In [13]:
def parse(s):
    ret="{\n"
    i1=s.find('=',0)
    i2=s.find('&',0)
    prev_v=0
    while(i1!=-1 and i2!=-1):
        s1=s[prev_v:i1]
        s2=s[i1+1:i2]
        prev_v=i2+1
        ret+="\t"+s1+": '"+s2+"',\n"
        i1=s.find('=',i2+1)
        i2=s.find('&',i2+1)
    if(i1!=-1):
        s1=s[prev_v:i1]
        s2=s[i1+1:]
        ret+="\t"+s1+": '"+s2+"',\n"
    ret=ret[:-2]
    ret+='\n}'
    return ret

In [15]:
print(parse("foo=bar&baz=qux&zap=zazzle"))
print(parse("email=foo@bar.com&uid=10&role=user"))

{
	foo: 'bar',
	baz: 'qux',
	zap: 'zazzle'
}
{
	email: 'foo@bar.com',
	uid: '10',
	role: 'user'
}
